In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))

from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql.functions import udf, monotonically_increasing_id, first, split

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.sparkContext.setCheckpointDir('.checkpoints')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/24 07:49:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Helper Functions

def concat_dataframes(dataframes: list):
    out = dataframes.pop()
    for df in dataframes:
        out = out.union(df)
    return out

def rename_columns(dataframe, col_name_map:dict[str, str]):
    for col in col_name_map.keys():
        dataframe = dataframe.withColumnRenamed(col, col_name_map[col])
    return dataframe

def convert_df_datatypes(df, types:dict[str, list[str]]):
    if isinstance(next(iter(types.values())), str): # types expects {'type': ['col1', ...]}
        raise TypeError("types expects list of columns {'type': ['col1', ...]}")
    types = {types[k][i]: k for k in types.keys() for i in range(len(types[k]))}
    return df.withColumns({k: df[k].cast(types[k]) for k in types})

def split_csv_column(dataframe, csv_col:str):
    col_count = dataframe.head().asDict()[csv_col].count(',') + 1
    split_col = split(dataframe[csv_col], ',')
    return dataframe.withColumns({f'csv{i}': split_col.getItem(i) for i in range(col_count)})

## Source Data - Events

In [4]:
DATA_PATH = '/Users/morgan/Documents/Codes/mlbEventData/data/retrosheet/'
EVENT_DIRS = ['events/', 'ngl_b/', 'ngl_e/', 'allstar/']


In [5]:
schema = types.StructType([
    types.StructField('event_type', types.StringType(), False),
    types.StructField('col2', types.StringType()),
    types.StructField('col3', types.StringType()),
    types.StructField('col4', types.StringType()),
    types.StructField('col5', types.StringType()),
    types.StructField('col6', types.StringType()),
    types.StructField('col7', types.StringType()),
])

In [6]:
# import all events
events = [spark.read.csv(DATA_PATH + ed, schema=schema) for ed in EVENT_DIRS]
events = concat_dataframes(events)

In [7]:
# count of games
events.filter(events.event_type == 'id').count()

201693

In [8]:
# used in udf
class GameId:

    def __init__(self):
        self.current_id = None

    def set_row_id(self, event_type, col2):
        if event_type=='id':
            self.current_id = col2
        if not self.current_id:
            raise ValueError('current_id cannot be None.')
        return self.current_id

In [9]:
gi = GameId()
set_game_id = udf(gi.set_row_id)

events = events.withColumns({'game_id': set_game_id(events['event_type'], events['col2']),
                            'record_id': monotonically_increasing_id()})
events = events.checkpoint()  # checkpoint locks game_id & record_id

In [10]:
events.show(20)

+----------+-------------+--------------------+----+----+----+----+------------+---------+
|event_type|         col2|                col3|col4|col5|col6|col7|     game_id|record_id|
+----------+-------------+--------------------+----+----+----+----+------------+---------+
|        id| ALS196007110|                NULL|NULL|NULL|NULL|NULL|ALS196007110|        0|
|   version|            1|                NULL|NULL|NULL|NULL|NULL|ALS196007110|        1|
|      info|inputprogvers|version 7RS(19) o...|NULL|NULL|NULL|NULL|ALS196007110|        2|
|      info|      visteam|                 NLS|NULL|NULL|NULL|NULL|ALS196007110|        3|
|      info|     hometeam|                 ALS|NULL|NULL|NULL|NULL|ALS196007110|        4|
|      info|         date|          1960/07/11|NULL|NULL|NULL|NULL|ALS196007110|        5|
|      info|         site|               KAN05|NULL|NULL|NULL|NULL|ALS196007110|        6|
|      info|       number|                   0|NULL|NULL|NULL|NULL|ALS196007110|        7|

In [11]:
events.dtypes

[('event_type', 'string'),
 ('col2', 'string'),
 ('col3', 'string'),
 ('col4', 'string'),
 ('col5', 'string'),
 ('col6', 'string'),
 ('col7', 'string'),
 ('game_id', 'string'),
 ('record_id', 'bigint')]

In [12]:
# events.limit(1000).write.csv('out/test.csv')

In [13]:
events.count()

30709643

## Game Info Table

In [14]:
type_map = {'short': ['number', 'temp', 'timeofgame', 'windspeed'],
            'integer': ['attendance'],
            'date': ['date'],
            'timestamp': ['inputtime']}
# also works:
# type_map = {types.ShortType(): ['number', 'temp', 'timeofgame', 'windspeed'],
#             types.IntegerType(): ['attendance'],
#             types.DateType(): ['date'],
#             types.TimestampType(): ['inputtime']}

game_info_table = (events.filter(events.event_type == 'info')
                         .select('game_id', 'col2', 'col3')
                         .groupBy('game_id').pivot('col2').agg(first('col3')))
game_info_table = convert_df_datatypes(game_info_table, types=type_map)

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=35674Kb max_used=35674Kb free=95397Kb
 bounds [0x00000001089f8000, 0x000000010ad18000, 0x00000001109f8000]
 total_blobs=13482 nmethods=12517 adapters=876
 compilation: disabled (not enough contiguous free space left)


In [15]:
game_info_table.show()

25/02/24 07:49:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+----------+----+--------+--------+---------+------------+-----+--------+----------+---------+----+-------+--------------------+--------------------+---------+--------+------+--------+-------+-------+--------+------------------+-----+--------+---------+----+----------+----------+--------------------+--------+--------+--------+--------+--------+--------+-----+-------+-------+---------+--------+
|     game_id|attendance|date|daynight|edittime|fieldcond|    gametype|gwrbi|hometeam|howentered|howscored|htbf|innings|       inputprogvers|            inputter|inputtime|      lp|number| oscorer|pitches| precip|    save|            scorer| site|     sky|starttime|temp|tiebreaker|timeofgame|          translator|   ump1b|   ump2b|   ump3b| umphome|   umplf|   umprf|usedh|visteam|winddir|windspeed|      wp|
+------------+----------+----+--------+--------+---------+------------+-----+--------+----------+---------+----+-------+--------------------+--------------------+---------+--------

In [16]:
game_info_table.dtypes

[('game_id', 'string'),
 ('attendance', 'int'),
 ('date', 'date'),
 ('daynight', 'string'),
 ('edittime', 'string'),
 ('fieldcond', 'string'),
 ('gametype', 'string'),
 ('gwrbi', 'string'),
 ('hometeam', 'string'),
 ('howentered', 'string'),
 ('howscored', 'string'),
 ('htbf', 'string'),
 ('innings', 'string'),
 ('inputprogvers', 'string'),
 ('inputter', 'string'),
 ('inputtime', 'timestamp'),
 ('lp', 'string'),
 ('number', 'smallint'),
 ('oscorer', 'string'),
 ('pitches', 'string'),
 ('precip', 'string'),
 ('save', 'string'),
 ('scorer', 'string'),
 ('site', 'string'),
 ('sky', 'string'),
 ('starttime', 'string'),
 ('temp', 'smallint'),
 ('tiebreaker', 'string'),
 ('timeofgame', 'smallint'),
 ('translator', 'string'),
 ('ump1b', 'string'),
 ('ump2b', 'string'),
 ('ump3b', 'string'),
 ('umphome', 'string'),
 ('umplf', 'string'),
 ('umprf', 'string'),
 ('usedh', 'string'),
 ('visteam', 'string'),
 ('winddir', 'string'),
 ('windspeed', 'smallint'),
 ('wp', 'string')]

In [17]:
game_info_table.count()

200408

## Game Data Table

In [18]:
game_data_table = events.filter(events.event_type == 'data').select(['game_id', 'col2', 'col3', 'col4'])
game_data_table = rename_columns(game_data_table, 
                                 {'col2': 'metric_type', 'col3': 'player_code', 'col4': 'metric_value'})
game_data_table = convert_df_datatypes(game_data_table, {'short': ['metric_value']})

In [19]:
game_data_table.show()

+------------+-----------+-----------+------------+
|     game_id|metric_type|player_code|metric_value|
+------------+-----------+-----------+------------+
|ALS196007110|         er|   frieb102|           0|
|ALS196007110|         er|   mccom103|           0|
|ALS196007110|         er|   facer101|           0|
|ALS196007110|         er|   buhlb101|           1|
|ALS196007110|         er|   law-v101|           0|
|ALS196007110|         er|   monbb101|           4|
|ALS196007110|         er|   estrc101|           1|
|ALS196007110|         er|   coatj101|           0|
|ALS196007110|         er|   bellg102|           0|
|ALS196007110|         er|   laryf101|           0|
|ALS196007110|         er|   daleb102|           0|
|ALS196007130|         er|   law-v101|           0|
|ALS196007130|         er|   podrj101|           0|
|ALS196007130|         er|   wills102|           0|
|ALS196007130|         er|   jackl102|           0|
|ALS196007130|         er|   henrb101|           0|
|ALS19600713

In [20]:
game_data_table.count()

1143314

## Game Rosters Table

In [21]:
game_rosters_table = (events.filter(events.event_type.isin(['start', 'sub']))
                            .select(['game_id', 'record_id', 'event_type', 
                                     'col2', 'col3', 'col4', 'col5', 'col6']))
game_rosters_table = rename_columns(game_rosters_table,
                                    {'col2': 'player_id', 'col3': 'player_name', 
                                     'col4': 'is_home_team', 'col5': 'batting_order',
                                     'col6': 'fielding_position'})
game_rosters_table = convert_df_datatypes(game_rosters_table,
                                          {'boolean': ['is_home_team'],
                                           'short': ['batting_order', 'fielding_position']})

In [22]:
game_rosters_table.show()

+------------+---------+----------+---------+-----------------+------------+-------------+-----------------+
|     game_id|record_id|event_type|player_id|      player_name|is_home_team|batting_order|fielding_position|
+------------+---------+----------+---------+-----------------+------------+-------------+-----------------+
|ALS196007110|       36|     start| maysw101|      Willie Mays|       false|            1|                8|
|ALS196007110|       37|     start| skinb101|      Bob Skinner|       false|            2|                7|
|ALS196007110|       38|     start| mathe101|    Eddie Mathews|       false|            3|                5|
|ALS196007110|       39|     start| aaroh101|       Hank Aaron|       false|            4|                9|
|ALS196007110|       40|     start| banke101|      Ernie Banks|       false|            5|                6|
|ALS196007110|       41|     start| adcoj101|       Joe Adcock|       false|            6|                3|
|ALS196007110|     

In [23]:
#TODO: dtypes

## Umpire Change Events Table

In [24]:
umpchange_events_table = events.filter(events.col2.contains('umpchange'))
umpchange_events_table = split_csv_column(umpchange_events_table, 'col2')
umpchange_events_table = umpchange_events_table.select(['record_id', 'game_id', 
                                                        'csv1', 'csv2', 'csv3'])
umpchange_events_table = rename_columns(umpchange_events_table, {'csv1': 'inning',
                                                                 'csv2': 'position',
                                                                 'csv3': 'umpire_id'})
umpchange_events_table = convert_df_datatypes(umpchange_events_table, {'short': ['inning']})

In [25]:
umpchange_events_table.show()

+---------+------------+------+--------------+---------+
|record_id|     game_id|inning|      position|umpire_id|
+---------+------------+------+--------------+---------+
|      115|ALS196007110|     5|       umphome| boggd901|
|      116|ALS196007110|     5|ump1b.chyln901|     NULL|
|      117|ALS196007110|     5|         ump2b| gormt101|
|      118|ALS196007110|     5|         ump3b| honoj901|
|      330|ALS196007130|     5|       umphome| gormt101|
|      331|ALS196007130|     5|         ump1b| honoj901|
|      332|ALS196007130|     5|         ump2b| boggd901|
|      333|ALS196007130|     5|         ump3b| chyln901|
|      938|NLS195907070|     5|       umphome| papaj901|
|      939|NLS195907070|     5|         ump1b| donaa901|
|      940|NLS195907070|     5|         ump2b| runge901|
|      941|NLS195907070|     5|         ump3b| barla901|
|     1147|NLS195908030|     5|       umphome| summb901|
|     1148|NLS195908030|     5|         ump1b| venzt901|
|     1149|NLS195908030|     5|

In [26]:
umpchange_events_table.count()

1582

In [27]:
umpchange_events_table.dtypes

[('record_id', 'bigint'),
 ('game_id', 'string'),
 ('inning', 'smallint'),
 ('position', 'string'),
 ('umpire_id', 'string')]

## Comments Table

In [28]:
comments_table = (events.filter(events.event_type == 'com')
                              .filter(~events.col2.contains('umpchange'))
                              .select(['record_id', 'game_id', 'col2']))

In [29]:
comments_table.show(20, False)

+---------+------------+---------------------------------------------------------------------+
|record_id|game_id     |col2                                                                 |
+---------+------------+---------------------------------------------------------------------+
|54       |ALS196007110|$Managers: Walter Alston (NL Los Angeles), Al Lopez (AL Chicago);    |
|55       |ALS196007110|Rosters 30, not 25; starters chosen by managers, players             |
|56       |ALS196007110|and coaches; no voting for own team; reserves and pitchers           |
|57       |ALS196007110|chosen by Alston and Lopez.                                          |
|58       |ALS196007110|$Weather: temps reached 100 degrees by the 8th inning.               |
|119      |ALS196007110|$The umpires changed positions                                       |
|205      |ALS196007110|$Boyer reached 3B but returned to 2B because of fan interference     |
|280      |ALS196007130|$Managers: Walter Alston (

In [30]:
comments_table.count()

234070

## Adjustments Table

In [31]:
@udf(returnType=types.MapType(types.StringType(), types.StringType()))
def _adjustment_values_map(event_type, col2, col3):
    if event_type == 'badj':
        out = {'event_name': 'Batter Adjustment','player_id': col2, 'hand': col3}
    elif event_type == 'padj':
        out = {'event_name': 'Pitcher Adjustment', 'player_id': col2, 'hand': col3}
    elif event_type == 'ladj':
        out = {'event_name': 'Lineup Adjustment', 'batting_team': col2, 'batting_order_position': col3}
    elif event_type == 'radj':
        out = {'event_name': 'Runner Adjustment', 'player_id': col2, 'base': col3}
    elif event_type == 'presadj':
        out = {'event_name': 'Pitcher Responsiblity Adjustment', 'player_id': col2, 'base': col3}
    else:
        out = None
    return out


In [32]:
adjustments_table = events.filter(events.event_type.isin(['badj', 'padj', 'ladj', 'radj', 'presadj']))
adjustments_table = (adjustments_table.withColumn('adjustment_events_map',
                                                  _adjustment_values_map(adjustments_table.event_type,
                                                                         adjustments_table.col2,
                                                                         adjustments_table.col3))
                                      .select(['record_id', 'game_id', 
                                               'event_type', 'adjustment_events_map']))

In [33]:
adjustments_table.show(20, False)

+-----------+------------+----------+----------------------------------------------------------------------------------+
|record_id  |game_id     |event_type|adjustment_events_map                                                             |
+-----------+------------+----------+----------------------------------------------------------------------------------+
|68719492263|DET194406160|presadj   |{event_name -> Pitcher Responsiblity Adjustment, player_id -> orrej101, base -> 3}|
|68719493840|DET194408272|presadj   |{event_name -> Pitcher Responsiblity Adjustment, player_id -> eatoz101, base -> 3}|
|68719541035|PHA194505240|ladj      |{batting_order_position -> 6, event_name -> Lineup Adjustment, batting_team -> 1} |
|68719541038|PHA194505240|ladj      |{batting_order_position -> 5, event_name -> Lineup Adjustment, batting_team -> 1} |
|68719541047|PHA194505240|ladj      |{batting_order_position -> 6, event_name -> Lineup Adjustment, batting_team -> 1} |
|68719545667|MIA202403280|radj  

In [34]:
adjustments_table.dtypes

[('record_id', 'bigint'),
 ('game_id', 'string'),
 ('event_type', 'string'),
 ('adjustment_events_map', 'map<string,string>')]

In [35]:
adjustments_table.count()

8227

## Events Table

In [36]:
events_table = (events.filter(events.event_type == 'play')
                      .select(['record_id', 'game_id', 'col2', 'col3', 
                               'col4', 'col5', 'col6', 'col7']))
col_names = {'col2': 'inning', 'col3': 'is_home_team', 'col4': 'player_id', 
             'col5': 'count', 'col6': 'pitches', 'col7': 'play_description'}
events_table = rename_columns(events_table, col_names)
events_table = convert_df_datatypes(events_table, {'short': ['inning'], 
                                                   'boolean': ['is_home_team']})

In [37]:
events_table.show()

+---------+------------+------+------------+---------+-----+--------+----------------+
|record_id|     game_id|inning|is_home_team|player_id|count| pitches|play_description|
+---------+------------+------+------------+---------+-----+--------+----------------+
|       59|ALS196007110|     1|       false| maysw101|   10|      BX|          T9/L9L|
|       60|ALS196007110|     1|       false| skinb101|   00|       X|      S8/L78.3-H|
|       61|ALS196007110|     1|       false| mathe101|   02|     CCX|       3/P23F/FL|
|       62|ALS196007110|     1|       false| aaroh101|   10|     B>B|             SB2|
|       63|ALS196007110|     1|       false| aaroh101|   32|B>B.BSSX|          8/F89S|
|       64|ALS196007110|     1|       false| banke101|   02|     CFX|    HR/F78XD.2-H|
|       65|ALS196007110|     1|       false| adcoj101|   00|       X|           9/F9D|
|       66|ALS196007110|     1|        true| minom101|   12|   BCFFX|          8/F78S|
|       67|ALS196007110|     1|        true

In [38]:
events_table.dtypes

[('record_id', 'bigint'),
 ('game_id', 'string'),
 ('inning', 'smallint'),
 ('is_home_team', 'boolean'),
 ('player_id', 'string'),
 ('count', 'string'),
 ('pitches', 'string'),
 ('play_description', 'string')]

In [39]:
events_table.count()

17467488

## Test Write to Database

In [40]:
# from my_secrets import airflow_pgres_pw


# # db_writer = DataFrameWriter(game_info_table)
# db_url = 'jdbc:postgresql://localhost:5432/sports'
# table = 'test.test_spark'
# mode = 'overwrite'
# properties = {"user": "airflow", 'password': airflow_pgres_pw, 
#               'driver': 'org.postgresql.Driver'}
# game_data_table.write.jdbc(url=db_url, table=table, mode=mode,
#                            properties=properties)